## Recommender System - Marketplace Matching

In this notebook, we will: 
- Clean textual data from user-input verbatim posts
- Use a Word2Vec model to calculate document similarities 
- Sort the most similar user input to our training data in order to recommend similar products 
- Save this model in a format that allows us to refresh the testing data

The goal of this project is to create a recommender system to help Pangeans find the right "project" for them, given their profile information. Here, we are using legacy data that is from Pangea V2, when Pangeans were allowed to post services and requests, as well as purchase items on the platform. We are using the User-Inputted Titles to suggest similar services, or in V3, similar "projects". 

In [1]:
#Importing Libraries
import numpy as np
import pandas as pd

import sys
from pandas import DataFrame

import json
from pandas.io.json import json_normalize
import csv

import matplotlib.pyplot as plt
%matplotlib inline 
from matplotlib import colors
from matplotlib.ticker import PercentFormatter

from gensim import corpora
from collections import defaultdict
from pprint import pprint

from gensim.models.word2vec import Word2Vec
from gensim.test.utils import common_texts, get_tmpfile

from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from gensim.test.utils import common_texts

import re
from sklearn.feature_extraction.text import CountVectorizer

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize 
nltk.download('stopwords') ###

import gensim 
from gensim.models import KeyedVectors
import gensim.downloader as api

from operator import itemgetter, attrgetter

from gensim.models.doc2vec import Doc2Vec

import os, sys

from operator import add

from sklearn.preprocessing import normalize

import seaborn as sns

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

from adjustText import adjust_text

from sklearn.manifold import TSNE

import pickle

import json
from pprint import pprint
from pandas.io.json import json_normalize

import operator

import csv

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/angelateng/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
model = gensim.models.KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin', binary = True)

In [3]:
def vectorize_and_store_existing_titles(): 
    raw = pd.read_csv("allPostData.csv", header=0);
    titles = raw['title'];
    post_titles = [title for title in titles];
    post_titles = set(post_titles);
    tokens = [[word for word in title.lower().split()] for title in post_titles];
    clean_words = [[word for word in title if word.isalpha()] for title in tokens];
    stoplist = set(stopwords.words('english'));
    titles_nostopwords = [[word for word in title if word not in stoplist] for title in clean_words];   
    filtered_word_list = [[word for word in title if word in model.vocab] for title in titles_nostopwords];
    dictionary = dict(zip(post_titles, filtered_word_list))
    vectorized_titles = pd.DataFrame(columns=["Titles", "Vectors"])
    for title in post_titles: 
        word_vecs = [model[word] for word in dictionary[title]]
        if len(word_vecs) == 0:
            title_vec = [np.zeros(300)]
        else: 
            title_vec = normalize(sum(word_vecs).reshape(1, -1))
        vectorized_titles = vectorized_titles.append({'Titles': title, 'Vectors': title_vec}, ignore_index=True)
    vectorized_titles.to_pickle("/Users/angelateng/Dropbox/AMBER/SharpestMinds/vectorized_titles.pkl")
    return(vectorized_titles)


In [4]:
vectorize_and_store_existing_titles()

,Titles,Vectors
0,CS homework help,"[[-0.040110506, 0.058018614, 0.042988595, 0.01..."
1,Take photos at BBQ,"[[-0.09394559, 0.0686004, 0.043736313, 0.07938..."
2,Personal pet for a day,"[[0.015168878, 0.05263752, -0.04207978, 0.0673..."
3,Animator,"[[0.03662849, -0.092960574, -0.045975067, 0.10..."
4,Photography / Portraits,"[[0.037897132, 0.075802274, -0.016601766, 0.04..."
5,Zero Waste Pop Up Shop,"[[-0.0057765376, 0.05752307, 0.058385674, 0.14..."
6,Family portrait,"[[0.04924191, 0.015757412, -0.07360962, 0.0505..."
7,Delicious Cookies,"[[-0.040297356, -0.052954573, 0.0010286784, 0...."
8,Bate Papo/Portuguese Lessons,"[[0.05909906, 0.089635506, 0.0881261, 0.029143..."
9,LaTex Tutor,"[[0.0619012, -0.0149769895, -0.07452506, 0.088..."


In [8]:
with open('firstPost.json') as fresh_data:
    user_post = json.load(fresh_data)

def vectorize_new_title(user_post):
    json_df = pd.DataFrame.from_dict(json_normalize(user_post), orient='columns')
    title = json_df["title"][0] #--> added [0] because json_df["title"] was still in pd.Series form
    json_tokens = [word for word in title.lower().split()]
    json_clean_words = [word for word in json_tokens if word.isalpha()]
    stoplist = set(stopwords.words('english'));
    json_titles_nostopwords = [word for word in json_clean_words if word not in stoplist]   
    json_preprocessed = [word for word in json_titles_nostopwords if word in model.vocab]   
    json_title_vectors = {}
    json_vectorized_title_df = pd.DataFrame(columns=["Titles", "Vectors"])
    json_word_vecs = [model[word] for word in json_preprocessed]
    if len(json_preprocessed) == 0:
            json_title_vec = [np.zeros(300)]
    else: 
        json_title_vec = normalize(sum(json_word_vecs).reshape(1, -1))
    json_vectorized_title_df = json_vectorized_title_df.append({'Titles': title, 'Vectors': json_title_vec}, ignore_index = True)
    return(json_vectorized_title_df)

In [9]:
vectorized_title = vectorize_new_title(user_post)
#print(vectorized_title)

def rank_existing_titles(vectorized_title):
    #loop over all keys in dict 
    ranked_titles = {}
    #other_titles = pd.read_pickle("/Users/angelateng/Dropbox/SharpestMinds/raw_titles.pkl")
    other_titles = pd.read_pickle("/Users/angelateng/Dropbox/AMBER/SharpestMinds/vectorized_titles.pkl")
   
    #can also use title_vectors.keys() 
    for index,row in other_titles.iterrows():
        ranked_titles[row['Titles']] = np.dot(np.squeeze(np.asarray(row['Vectors'])), np.squeeze(np.asarray(vectorized_title['Vectors'][0])))
        #print(ranked_titles[row['Titles']])
        
    sorted_title_vecs = sorted(ranked_titles.items(), key=operator.itemgetter(1), reverse=True)
    return(sorted_title_vecs)

#rank_existing_titles(vectorized_title[:10])
#word querying

#should be cleaned titles x cleaned til

In [10]:
vectorized_title = vectorize_new_title(user_post)

def rank_existing_titles(vectorized_title):
    #loop over all keys in dict 
    ranked_titles = {}
    other_titles = pd.read_pickle("./vectorized_titles.pkl")
    for index,row in other_titles.iterrows():
        ranked_titles[row['Titles']] = sum(row['Vectors'][0]*vectorized_title['Vectors'][0][0]) # --> did the dot product using sum() and * because np.dot was behaving weirdly for some reason. Now it seems to work! 
    sorted_title_vecs = sorted(ranked_titles.items(), key=operator.itemgetter(1), reverse=True)
    return(sorted_title_vecs)

In [12]:
def generate_recommendations(user_post):
    title = user_post["title"]
    vectorized_title = vectorize_new_title(user_post)

    ranked_titles = rank_existing_titles(vectorized_title)
    
    with open("./ranked_titles.csv", "w", newline='') as myfile:
        wr = csv.writer(myfile, quoting=csv.QUOTE_ALL)
        for title in ranked_titles:
            wr.writerow([ranked_titles, title])
    #csv_out = ranked_titles.to_csv('ranked_titles', encoding='utf-8', index=False)
    #writer = csv.writer(ranked_titles, delimiter='', quotechar='"', quoting=csv.QUOTE_ALL)
 
    #for row in reader:
        #writer.writerow(row)

       # with open('csv_out', 'wb') as csv_out:
         #   csv_writer = csv.writer(csv_out, quoting=csv.QUOTE_ALL)
         #   csv_writer.writerow('csv_out')
    return(ranked_titles)
    

In [13]:
#generate_recommendations(json_df)
pprint(generate_recommendations(user_post))

[('Teach Me How To Cook!', 1.0000000026004443),
 ('Teach you how to shuffle', 0.7285079273577821),
 ('Teach me how to cook quick meals!', 0.7162580516296657),
 ('Teach me how to play piano', 0.7146377012308847),
 ('Teach how to make tea', 0.6914844676543908),
 ('Learn To Juggle!', 0.6698537354463951),
 ('Teach how to prepare steaks ', 0.6608135595374733),
 ('Help me learn Pop/Stars dance', 0.6424254786616075),
 ('Teach Me How To Solve Rubiks Cube', 0.6324938975573104),
 ('Teach me meme culture', 0.6134594880758186),
 ('Orgo lesson ', 0.5588066486525349),
 ('Math tutor', 0.5531746304923217),
 ('Math Tutor', 0.5531746304923217),
 ('Cooking Lessons', 0.5473080828668344),
 ('Cooking Lessons ', 0.5473080828668344),
 ('Cooking lessons', 0.5473080828668344),
 ('Learn To Harmonize', 0.5445825279987275),
 ('Prototyping/Wireframing Lessons', 0.5328233039076622),
 ('Fortnite Lessons', 0.5328233039076622),
 ('Amharic lessons ', 0.5328233039076622),
 ('Self-Defense Lessons', 0.5328233039076622),
 (